#### Import / Install packages

###### Use Conda to install the wikipedia and pandasql packages if needed 

In [ ]:
#Uncomment these if needed
!conda install -c conda-forge wikipedia --yes
!conda install -c conda-forge pandasql --yes

Solving environment: / 

###### Import the packages we need and set pysql to use global variables 

In [ ]:
import pandas as pd
import wikipedia as wp
import numpy as np
import pandasql as pdsql
pysql = lambda q: pdsql.sqldf(q, globals())

#### Read Page into Dataframe

In [ ]:
#Get the html source
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]

###### Remove cells with a borough that is Not assigned

In [ ]:
pysql = lambda q: pdsql.sqldf(q, globals())
df=pysql('select * from df where borough!=\'Not assigned\'')

###### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [ ]:
df=pysql('select Postcode, Borough, CASE when Neighbourhood=\'Not assigned\' then Borough else Neighbourhood END as Neighbourhood from df order by Postcode, Borough')
#df3

###### Combine Postcode / Boroughs with multiple neighborhoods into one row with the neighborhoods separated with a comma

In [ ]:
# Manipulate data in a numpy array
x=df.to_numpy()
y=int(len(df.index))-1
z=0
q=z-1
while z<y:
    while q<=(y-1):
        if x[z,0]==x[q,0] and x[z,1]==x[q,1] and x[z,2]!=x[q,2]:
            W=x[q,2] + "," + x[z,2]
            x[z,2]=W
            x[q,2]=W
        q=q+1
        z=z+1
# Put data back in dataframe
df=pd.DataFrame({'Postcode': x[:, 0], 'Borough': x[:, 1], 'Neighbourhood': x[:, 2]})
#Query insures we get accurate multi-neighborhood records
df=pysql('select distinct Postcode, Borough, Neighbourhood from df where length(Neighbourhood)=(select max(length(Neighbourhood)) from df as q where df.Postcode=q.Postcode and df.Borough=q.Borough)')

###### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [ ]:
df.shape